In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
URL = "https://kingfisher.kz/"

In [5]:
HEADERS = ({'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'})

In [7]:
webpage = requests.get(URL, headers = HEADERS)

In [8]:
type(webpage.content)

bytes

In [11]:
soup = BeautifulSoup(webpage.content, "html.parser")

In [13]:
links = soup.find_all("a", attrs={'class' : 'title'})

In [15]:
link = links[0].get('href')

In [17]:
product_list = "https://kingfisher.kz" + link

In [19]:
product_list

'https://kingfisher.kz/ryba/ohlazhdennaya-ryba/sibas-ohlazhdenniy-vesom-ot-1000-do-1500-gr-greciya/'

In [21]:
new_webpage = requests.get(product_list, headers = HEADERS)

In [22]:
new_webpage

<Response [200]>

In [25]:
new_soup = BeautifulSoup(new_webpage.content, "html.parser")

In [27]:
new_soup.find("h1", attrs={"itemprop" : 'name'}).text

'Сибас охлажденный весом от 1000 до 1500 гр Турция'

In [29]:
new_soup.find("div", attrs={"class" : 'gPrice gPrice1'}).find("div", attrs={"class" : 'new'}).text

'12 990 тг.'

In [31]:
new_soup.find("div", attrs={"class" : 'gPrice gPrice2'}).find("div", attrs={"class" : 'new'}).text

'19 485 тг.'

In [33]:
rating_value = new_soup.find("div", attrs={"itemtype" : 'http://schema.org/AggregateRating'}).find("meta", attrs={"itemprop" : 'ratingValue'})["content"]

rating_value = int(rating_value)

rating_value

3

In [35]:
results = new_soup.find_all("span", attrs={'itemprop': 'name'})

if len(results) > 1:
    second_result = results[1].get_text()
else:
    second_result = None

print(second_result)

Рыба


In [38]:
def get_title(soup):

    try:
        title = soup.find("h1", attrs={"itemprop" : 'name'})
        
        title_value = title.text

        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

In [40]:
def get_category(soup):

    try:
        category = soup.find_all("span", attrs={'itemprop': 'name'})
        
        if len(category) > 1:
            category_string = category[1].get_text()

    except AttributeError:
        category_string = ""

    return category_string

In [42]:
def get_subcategory(soup):

    try:
        subcategory = soup.find_all("span", attrs={'itemprop': 'name'})
        
        if len(subcategory) > 2:
            subcategory_string = subcategory[2].get_text()

    except AttributeError:
        subcategory_string = ""

    return subcategory_string

In [55]:
def get_price(soup):

    try:
        price = soup.find("div", attrs={"class" : 'gPrice gPrice1'}).find("div", attrs={"class" : 'new'}).string.strip()
        price = price.replace(" тг.", "").replace(" ", "")
        price = int(price)
    except AttributeError:

        try:
            price = soup.find("div", attrs={"class" : 'gPrice gPrice1'}).find("div", attrs={"class" : 'new'}).string.strip()

        except:
            price = ""

    return price

In [57]:
def get_rating(soup):

    try:
        rating = soup.find("div", attrs={"itemtype" : 'http://schema.org/AggregateRating'}).find("meta", attrs={"itemprop" : 'ratingValue'})["content"]
        rating = int(rating)
    except AttributeError:
        try:
            rating = soup.find("div", attrs={"itemtype" : 'http://schema.org/AggregateRating'}).find("meta", attrs={"itemprop" : 'ratingValue'})["content"]
        except:
            rating = ""	

    return rating

In [61]:
categories = [
    "moreprodukty", "ryba", "ikra", "myaso", "antipasti", 
    "bakaleya", "sneki-i-orehi", "maslo", "cheeses"
]

d = {"title":[], "category":[], "subcategory": [], "price":[], "rating":[]}

In [63]:
for category in categories:
    URL = f"https://kingfisher.kz/{category}/"

    webpage = requests.get(URL, headers=HEADERS)

    soup = BeautifulSoup(webpage.content, "html.parser")

    links = soup.find_all("a", attrs={'class': 'title'})

    links_list = []

    print(f"https://kingfisher.kz/{category}/")

    for link in links:
        links_list.append(link.get('href'))

    for link in links_list:
        new_webpage = requests.get("https://kingfisher.kz" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        d['title'].append(get_title(new_soup))
        d['category'].append(get_category(new_soup))
        d['subcategory'].append(get_subcategory(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))

https://kingfisher.kz/moreprodukty/
https://kingfisher.kz/ryba/
https://kingfisher.kz/ikra/
https://kingfisher.kz/myaso/
https://kingfisher.kz/antipasti/
https://kingfisher.kz/bakaleya/
https://kingfisher.kz/sneki-i-orehi/
https://kingfisher.kz/maslo/
https://kingfisher.kz/cheeses/


In [64]:
kingfisher_df = pd.DataFrame.from_dict(d)

In [65]:
kingfisher_df

,title,category,subcategory,price,rating
0,Креветки Королевские 26/30 очищенные 1 кг,Морепродукты,Креветки,7500,5
1,Креветка Королевская 16/20 в панцире б/г 1 кг,Морепродукты,Креветки,7800,
2,Креветка Королевская 31/40 очищенная 1 кг/уп,Морепродукты,Креветки,7600,
3,"Креветка Королевская 21/25, в панцире без головы",Морепродукты,Креветки,7600,
4,"Лангустины в панцире с головой, размер 21-30 у...",Морепродукты,Креветки,11490,
...,...,...,...,...,...
772,"Ghidetti Сыр Grana Padano Dop 10-12 мес, 1 кг",Сыры,Итальянский сыр,17500,
773,Сыр Манчего полувыдержанный 2-3 месяца Vega Ma...,Сыры,Испанские сыры,25990,
774,Ghidetti Сыр твердый Gran Guardia 1 кг,Сыры,Итальянский сыр,14500,
775,"Luigi Guffanti Пьяве Веччо сыр полутвердый, кр...",Сыры,Итальянский сыр,27500,


In [74]:
kingfisher_df.to_csv(r'C:\Users\menis\Desktop\kingfisher\kingfisher_data.xls', header=True, index=False)

print('uploaded')

uploaded
